# Evaluate Pre-Trained DeepLabV3 Model Predictions

- Outputs CSV file and image predictions (in blue shading) for three DeepLabV3 model ensemble.

__Note:__ To maintain a high priority Colab user status such that sufficient GPU resources are available in the future, ensure to free the runtime when finished running this notebook. This can be done using 'Runtime > Manage Sessions' and click 'Terminate'.

In [ ]:
import os
import os.path as osp
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !ln - sf / opt/bin/nvidia-smi / usr/bin/nvidia-smi
    !pip install gputil
    !pip install psutil
    !pip install humanize
    
    # Check if notebook is running in Colab or local workstation
    import GPUtil as GPU
    import humanize
    import psutil

    GPUs = GPU.getGPUs()

    try:
        # XXX: only one GPU on Colab and isn’t guaranteed
        gpu = GPUs[0]

        def printm():
            process = psutil.Process(os.getpid())
            print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available),
                  " | Proc size: " + humanize.naturalsize(process.memory_info().rss))
            print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(
                gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
        printm()

        # Check if GPU capacity is sufficient to proceed
        if gpu.memoryFree < 10000:
            print("\nInsufficient memory! Some cells may fail. Please try restarting the runtime using 'Runtime → Restart Runtime...' from the menu bar. If that doesn't work, terminate this session and try again later.")
        else:
            print('\nGPU memory is sufficient to proceeed.')
    except:
        print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
        print('and then re-execute this cell.')

In [ ]:
if IN_COLAB:

    from google.colab import drive
    drive.mount('/content/drive')
    DATA_PATH = r'/content/drive/My Drive/Data'

    # cd into git repo so python can find utils
    %cd '/content/drive/My Drive/cciw-zebra-mussel/predict'

    sys.path.append('/content/drive/My Drive')

    # clone repo, install packages not installed by default
    !pip install pydensecrf

In [ ]:
import csv
import glob

# for manually reading high resolution images
import cv2

# deep learning libs
import torch
from torch import nn

# convert opencv image to torch tensor
from task_3_utils import img_to_nchw_tensor

# progress bar
from tqdm import tqdm  

In [ ]:
"""Confim that this cell prints "Found GPU, cuda". If not, select "GPU" as 
"Hardware Accelerator" under the "Runtime" tab of the main menu.
"""
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('Found GPU,', device)

# 1. Meta-parameters

In [ ]:
SEED = 1  # 2, 3

"""Set to True to save the model predictions in PNG format, 
otherwise proceed to predict biomass without saving images"""
SAVE_PREDICTIONS = True

# option 1, validation
root_path = '/scratch/ssd/gallowaa/cciw/VOCdevkit/Validation-v102-originals/'  # N=55
split = 'val' 
version = 'v102'
scale_percent = 100 # these quadrats already resized from 3000 to 2250 px square

# option 2, train
#root_path = '/scratch/ssd/gallowaa/cciw/VOCdevkit/Train-v120-originals/'  # N=152
#split = 'train'
#version = 'v120'
#scale_percent = 100 # these quadrats already resized from 3000 to 2250 px square

# option 3, test
#root_path = '/scratch/ssd/gallowaa/cciw/dataset_raw/unused-train-cropped/'  # N=189
#split = 'test'
#version = 'v110'
#scale_percent = 75

# 2. Load a pre-trained model checkpoint

In [ ]:
os.environ['DATA_PATH'] = '/scratch/gallowaa/'

if IN_COLAB:
    root = osp.join(
        DATA_PATH, 'Checkpoints/deeplabv3_resnet50_lr1e-01_wd5e-04_bs40_ep80_seed1')

else:
    root = osp.join(
        os.environ['DATA_PATH'], 'cciw/logs/cmp-dataset/train_v120/deeplabv3_resnet50/lr1e-01/wd5e-04/bs40/ep80/seed%d/checkpoint' % SEED)

ckpt_file = 'deeplabv3_resnet50_lr1e-01_wd5e-04_bs40_ep80_seed%d_epoch40.ckpt' % SEED

model_to_load = osp.join(root, ckpt_file)

print('Loading', model_to_load)

checkpoint = torch.load(model_to_load)

train_loss = checkpoint['trn_loss']
val_loss = checkpoint['val_loss']
print('==> Resuming from checkpoint..')
net = checkpoint['net']
last_epoch = checkpoint['epoch']
torch.set_rng_state(checkpoint['rng_state'])

# later appended to figure filenames
model_stem = ckpt_file.split('.')[0]

print('Loaded model %s trained to epoch ' % model_stem, last_epoch)
print(
    'Cross-entropy loss {:.4f} for train set, {:.4f} for validation set'.format(train_loss, val_loss))

sig = nn.Sigmoid()  # initializes a sigmoid function

net.eval()

# 3. Load dataset

In [ ]:
# for VOCdevkit root path
jpeg_files = glob.glob(osp.join(root_path, 'JPEGImages/') + '*.jpg')
png_files = glob.glob(osp.join(root_path, 'SegmentationClass/') + '*_crop.png')
png_files.sort()
print('Found %d PNG masks' % len(png_files))

# otherwise
if len(jpeg_files) == 0:
    jpeg_files = glob.glob(root_path + '*.jpg')
jpeg_files.sort()
print('Found %d JPEG images' % len(jpeg_files))

In [ ]:
if SAVE_PREDICTIONS:
    prediction_path = ''
    for t in root.split('/')[:-1]:
        prediction_path += t + '/'

    prediction_path = osp.join(prediction_path, 'predictions-lom-%s-%s' % (split, version))

    if not osp.exists(prediction_path):
        os.mkdir(prediction_path)

    # src is the training dataset, tgt is the testing dataset
    src = 'train_v120'
    tgt = split + '_' + version
    print(prediction_path)

# 4. Do prediction and save as soft-blue images

In [ ]:
# updated July 17, 2020 for LOM journal. Blend soft predictions with orig image.

for i in tqdm(range(len(jpeg_files))):

    image_stem = jpeg_files[i].split('/')[-1].split('.')[0]

    bgr_img = cv2.imread(osp.join(root_path, jpeg_files[i]))
    imgc = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB)

    width = int(imgc.shape[1] * scale_percent / 100)
    height = int(imgc.shape[0] * scale_percent / 100)
    imgc = cv2.resize(imgc, (width, height)) # resize image

    nchw_tensor = img_to_nchw_tensor(imgc, device)

    with torch.no_grad():
        pred = sig(net(nchw_tensor)['out']) # remove 'out' for vanilla FCN
    pred_np = pred.detach().cpu().numpy().squeeze()

    p = (pred_np * 255).astype('uint8')
    src2 = np.zeros((p.shape[0], p.shape[1], 3), np.uint8)
    src2[:, :, 2] = p
    dst = cv2.addWeighted(imgc, 0.75, src2, 0.5, 0.5)

    if SAVE_PREDICTIONS:
        filename = src + '-' + tgt + '__' + image_stem + '__' + model_stem
        out_file = osp.join(prediction_path, filename)
        cv2.imwrite(out_file + '_scale%d_preds.jpg' % scale_percent, cv2.cvtColor(dst, cv2.COLOR_RGB2BGR))

# 5. Do prediction and save pixel counts as CSV

In [ ]:
# this cell added June 5, 2020 for saving predictions.
csvfile = 'predictions_%s-%s_%s' % (split, version, model_stem) 
csvfile += '.csv'
print('Creating', csvfile)

with open(csvfile, 'w') as f:
    csvwriter = csv.writer(f, delimiter=',')
    csvwriter.writerow(['image', 'mussel_pixels', 'total_pixels'])
    
    for i in tqdm(range(len(jpeg_files))):

        image_stem = jpeg_files[i].split('/')[-1].split('.')[0]

        bgr_img = cv2.imread(osp.join(root_path, jpeg_files[i]))
        imgc = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB)
        
        # resize image
        width = int(imgc.shape[1] * scale_percent / 100)
        height = int(imgc.shape[0] * scale_percent / 100)
        imgc = cv2.resize(imgc, (width, height)) 

        nchw_tensor = img_to_nchw_tensor(imgc, device)

        with torch.no_grad():
            pred = sig(net(nchw_tensor)['out']) # remove 'out' for vanilla FCN
        pred_np = pred.detach().cpu().numpy().squeeze()
        
        csvwriter.writerow([image_stem, pred_np.round().sum(), np.prod(imgc.shape[:2])])